In [2]:
import requests
import pandas as pd
import bs4
import sqlite3
import csv
import matplotlib.pyplot as plt

# ===================== 爬蟲函式 ===================== #
def crawler():
    global news_data

    dateTag = objSoup.find_all("div", class_="wp-block-kadence-advancedheading")
    dynamic_blocks = objSoup.find_all("div", class_="wp-block-kadence-dynamichtml")

    titles, contents = [], []
    current_title = None

    # 按照 HTML 順序配對標題與內容
    for block in dynamic_blocks:
        cls = " ".join(block.get("class", []))
        text = block.get_text(strip=True)
        if "_5a54c2-26" in cls:  # 標題
            current_title = text
        elif "_eab4c7-ca" in cls and current_title:  # 內容
            titles.append(current_title)
            contents.append(text)
            current_title = None

    # 日期數量可能不同，取最短長度
    min_len = min(len(titles), len(contents), len(dateTag))

    news_data = []
    for i in range(min_len):
        date_text = dateTag[i].get_text(strip=True) if i < len(dateTag) else ""
        news_data.append({
            "Title": titles[i],
            "Text": contents[i],
            "Date": date_text
        })

    print(f"✅ 已使用爬蟲取得資料，共 {len(news_data)} 筆。")


# ===================== 匯出 CSV ===================== #
def toCSV():
    global csvFile
    if not news_data:
        print("❌ 沒有資料可儲存，請先執行爬蟲（選項1）")
        return

    csvFile = pd.DataFrame(news_data)
    csvFile.to_csv('News.csv', encoding='utf_8_sig', index=False)
    print(f"✅ 已把爬蟲資料儲存為 News.csv，共 {len(csvFile)} 筆。")


# ===================== 顯示爬蟲結果 ===================== #
def print_text():
    count = min(len(news_data), 10)
    for i in range(count):
        print("\n事實查核報告:", news_data[i]["Title"])
        print("\n內容:", news_data[i]["Text"])
        print("\n日期:", news_data[i]["Date"])
        print("========================================================================================")


# ===================== 顯示 CSV ===================== #
def printCSV():
    print(csvFile)


# ===================== 查詢資料庫 ===================== #
def load_Data():
    print("\n查詢資料庫裏的資料")
    results = conn.execute("SELECT * FROM News")
    for record in results:
        print("========================================================================================")
        print("Index =", record[0])
        print("\n標題 =", record[1])
        print("\n內容 =", record[2])
        print("\n日期 =", record[3])


# ===================== 刪除資料庫 ===================== #
def delete_sql():
    sql = '''DROP TABLE IF EXISTS News'''
    conn.execute(sql)
    sql = '''CREATE TABLE News(IndexNum INT, Title TEXT, Text TEXT, Date TEXT)'''
    conn.execute(sql)
    print("✅ 已刪除資料庫裏的資料（並重建空表）")


# ===================== 更新資料庫 ===================== #
def updateDB():
    fn = 'News.csv'
    try:
        with open(fn, encoding="utf-8") as csvFile:
            csvReader = csv.reader(csvFile)
            listCsv = list(csvReader)
            # Assuming the first row is a header, skip it
            csvData = listCsv[1:]
            for i, row in enumerate(csvData): # Add enumerate to generate IndexNum
                # The CSV has Title, Text, Date (3 columns) because index=False was used in to_csv
                # The database expects IndexNum, Title, Text, Date (4 columns)
                IndexSQL = i + 1 # Generate a 1-based index
                titleSQL = row[0]
                textSQL = row[1]
                dateSQL = row[2]
                x = (IndexSQL, titleSQL, textSQL, dateSQL)
                sql = '''INSERT INTO News(IndexNum, Title, Text, Date) VALUES(?,?,?,?)'''
                conn.execute(sql, x)
                conn.commit()
            print("✅ 已更新資料庫，共", len(csvData), "筆。")
    except FileNotFoundError:
        print("❌ 找不到 News.csv，請先執行選項3 儲存CSV。")
    except IndexError as e:
        print(f"❌ 錯誤！CSV檔案格式不正確，預期每行有至少3欄資料(Title, Text, Date)。詳細錯誤: {e}")
        print(f"問題發生的行: {row}")
    except Exception as e:
        print(f"❌ 更新資料庫時發生未知錯誤: {e}")


# ===================== 關閉資料庫 ===================== #
def closeDB():
    conn.close()
    print("✅ 已關閉資料庫連線。")


# ===================== 主程式 ===================== #
conn = sqlite3.connect("Group.db")
conn.execute('PRAGMA encoding="UTF-8";')

news_data = []

url = 'https://tfc-taiwan.org.tw/articles/report'
response = requests.get(url)
response.encoding = 'utf-8'
objSoup = bs4.BeautifulSoup(response.text, 'lxml')

while True:
    print("\n====================== 假新聞判斷系統 ========================")
    print("1. 爬蟲台灣事實查核中心的資料")
    print("2. 顯示爬蟲的資料")
    print("3. 儲存爬蟲資料為 CSV 檔")
    print("4. 顯示 News.csv 的資料")
    print("5. 刪除資料庫裏現有的資料")
    print("6. 更新資料庫裏的資料")
    print("7. 查詢資料庫裏的資料")
    print("8. Exit")
    print("=============================================================")
    try:
        menu_input = int(input("\n請選擇要使用的功能 (1~8): "))
    except ValueError:
        print("❌ 請輸入數字 1~8")
        continue

    if menu_input == 1:
        crawler()

    elif menu_input == 2:
        if news_data:
            print_text()
        else:
            print("❌ 尚未爬取資料，請先選擇 1")

    elif menu_input == 3:
        toCSV()

    elif menu_input == 4:
        try:
            printCSV()
        except NameError:
            print("❌ Error! 需先儲存為 CSV 檔 (選項3)")

    elif menu_input == 5:
        delete_sql()

    elif menu_input == 6:
        updateDB()

    elif menu_input == 7:
        try:
            load_Data()
        except Exception as e:
            print("❌ Error! 查詢資料庫失敗")
            print("詳細錯誤:", e)

    elif menu_input == 8:
        closeDB()
        print("👋 Exit")
        break

    else:
        print("❌ 請輸入 1~8 之間的數字")


====================== 假新聞判斷系統 ========================
1. 爬蟲台灣事實查核中心的資料
2. 顯示爬蟲的資料
3. 儲存爬蟲資料為 CSV 檔
4. 顯示 News.csv 的資料
5. 刪除資料庫裏現有的資料
6. 更新資料庫裏的資料
7. 查詢資料庫裏的資料
8. Exit

請選擇要使用的功能 (1~8): 1
✅ 已使用爬蟲取得資料，共 9 筆。

====================== 假新聞判斷系統 ========================
1. 爬蟲台灣事實查核中心的資料
2. 顯示爬蟲的資料
3. 儲存爬蟲資料為 CSV 檔
4. 顯示 News.csv 的資料
5. 刪除資料庫裏現有的資料
6. 更新資料庫裏的資料
7. 查詢資料庫裏的資料
8. Exit

請選擇要使用的功能 (1~8): 2

事實查核報告: 網傳「西芹粉防癌降三高」是假冒醫師名義的一頁式詐騙網站

內容: 近日網路流傳連結販售「有機西芹粉」並聲稱是由腎臟科醫師洪永祥與廠商聯合販售；經查證，這是冒用洪永祥醫師名義的一頁式詐騙網站。

日期: 發佈日期：2025-11-06

事實查核報告: 假冒「美國國土安全部」帳號 虛構川普將宣布台灣獨立

內容: 近日社群平台流傳一張「美國國土安全部新聞」X帳號貼文擷圖，宣稱川普將在本月稍晚宣布台灣獨立於中國。經查，發佈此訊息的X帳號不是美國國土安全部官方帳號，川普與美國官方近期並未發布傳言說法。

日期: 發佈日期：2025-11-05

事實查核報告: AI深偽影片盜用台大名醫名號  販售無效肺部治療藥物

內容: 近期TikTok平台有影片宣稱台灣肺癌權威、台大外科部主任陳晉興研發一款可逆轉肺功能的藥物。這其實是挪用陳晉興演講畫面、以深偽變造唇型及聲音說出產品廣告用語。

日期: 發佈日期：2025-11-05

事實查核報告: 幻想文誤導網友，誤信是川習會對話

內容: 美國總統川普與中國國家主席習近平10月30日在韓國釜山舉行雙邊會談。近日網路流傳「川普在川習會嗆習近平談台灣主權」；經查證，沒有證據顯示川習會曾提及台灣議題，且網傳文章的發布時間早於川習會，其內容純屬虛構創作。

日期: 發佈日期：2025-11-04

事實查核報告: 環境部光電板實驗 清水